In [ ]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import colorsys
from skimage import data, io, filters, feature, morphology, measure, exposure, draw, transform
from deskew import determine_skew
%matplotlib inline

from threading import Thread
import pygame as pg
import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def contourArea(contour):
    s = 0
    l = contour.shape[0]
    for i in range(l):
        if i == l-1:
            s+= contour[i][0]*contour[0][1] - contour[i][1]*contour[0][0]
        else:
            s += contour[i][0]*contour[i+1][1] - contour[i][1]*contour[i+1][0]
    return np.abs(s/2)

#na podstawie https://pyshine.com/How-to-play-piano-using-Python/
def play_notes(note, duration):
    time.sleep(duration)
    pg.mixer.Sound('sounds/{}.mp3'.format(note)).play()
    time.sleep(duration)

In [ ]:
def get_notes_heights(image, ax, min_r):
    #just filter out background and invert image
    otsu_threshold = filters.threshold_otsu(image) 
    binary = image <= otsu_threshold
    
    # sizes of matrixes could change in the future ( to improve precision of notes processing )
    horizontal_morphology_matrix = np.ones((1,5))
    vertical_morphology_matrix = np.ones((3,1))
    
    processed_image = morphology.erosion(binary,vertical_morphology_matrix) #delete lines
    processed_image = morphology.dilation(processed_image,vertical_morphology_matrix) #notes shape rebuilding
    processed_image = morphology.dilation(processed_image,vertical_morphology_matrix) #notes shape rebuilding
    processed_image = morphology.erosion(processed_image,horizontal_morphology_matrix) # maybe swap  
    
    if ax:
        ax.imshow(image, cmap=plt.cm.gray)
        
    contours = measure.find_contours(processed_image)
        
    notes_heights = []
    X = []
    Y = []
    
    min_note_area = (min_r/2)**2 * 3.14
    
    for contour in contours:
        centroid_x = np.mean(contour[:,1])
        centroid_y = np.mean(contour[:,0])
        area = contourArea(contour)        
        if area < min_note_area:
            continue
        Y.append(centroid_y)
        X.append(centroid_x)
        if ax:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=2)
            ax.plot(centroid_x,centroid_y,marker='.',color='red',markersize=4)
    
    sorted_list = sorted(range(len(X)),key=lambda x:X[x])
    notes_heights = [Y[i] for i in sorted_list]
    
    return notes_heights
    

    
def get_lines_heights(image,ax):
    #just filter out background and invert image
    otsu_threshold = filters.threshold_otsu(image) 
    binary = image <= otsu_threshold
    
    horizontal_morphology_matrix = np.ones((1,30))
    processed_image = morphology.erosion(binary,horizontal_morphology_matrix) #delete notes
      
    contours = measure.find_contours(processed_image)
        
    if ax:
        ax.imshow(image, cmap=plt.cm.gray)
        
    lines_heights = []

    for contour in contours:
        lines_heights.append(np.mean(contour[:,0]))
        if ax:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=2)
               
    return lines_heights


def classify_notes(image_name,ax=None):
    image = io.imread(image_name, as_gray = True)

    try:
        angle = determine_skew(image)
        rotated = transform.rotate(image,angle,resize=False, mode='edge')*255
    except:
        rotated = image
    
    lines_heights = get_lines_heights(rotated,ax)
    
    avg_space_between_lines=(lines_heights[-1]-lines_heights[0])/4
    bias = avg_space_between_lines/4 #to describe height interval of every note
    
    notes_heights = get_notes_heights(rotated,ax,avg_space_between_lines)
    
    plt.show()
    
    detected_notes = []
    possible_notes = ['e3','f3','g3','a4','b4','c4','d4','e4','f4','g4','a5','b5','c5','d5','e5','f5','g5']
    
    step = 0.5
    for note_height in notes_heights:
        multipler = 3.5
        for note in possible_notes:
            if lines_heights[-1] + multipler*avg_space_between_lines+bias >= note_height > lines_heights[-1] \
                + multipler*avg_space_between_lines-bias:
                detected_notes.append(note)
                break
            multipler-=step
    
    return detected_notes

  


In [ ]:
pg.mixer.init()
pg.init()

fig, axes = plt.subplots(1,2,figsize=(12,7))
image_name = "hard/5.png"

image = io.imread(image_name, as_gray = True)
axes[0].imshow(image, cmap=plt.cm.gray)

detected_notes = classify_notes(image_name,axes[1])

pg.mixer.set_num_channels(len(detected_notes))
th = {} #dictionary for each thread

for note in detected_notes:
    print(note, end=' ')
    th[note] = Thread(target = play_notes,args=(note,0.2))
    th[note].start()
    th[note].join()
print()
